In [ ]:
#This notebook contains a quick implementation of the robust pca algorithm.
#We solve the problem using the implementation of "The Augmented Lagrange Multiplier
#method for exact recovery of corrupted low-rank matrices"
#In the end, we also use a solver to compare times and overall performance.
#We expect to find much more promising results using the implemantation on the latter paper:
#"/although interior point methods normally take very few iterations t converge, they have difficulty in
#handling large matrices because of the complexity of computing the step direction is O(m^6), where m is the dimension of 
#matrix [...] generic interior point solvers are too limited for Robust PCA ".

#Note that the robust PCA is used in many context. Ours is the clique problem which deals with nxn symetric matrices
#. However this implementation works for m x n matrices.



In [ ]:
# The model
#D- a m x n matrix of data/variables. Usually D is nxn graph adjacency matrix.
# We seek to solve the problem min a||E||_1 + ||A||_* subject to E+A = D+I 
#a is a positive real number. 


In [1]:
#rpca is the main function. 
#If D is square matrix, we redefine D to be D = D+I.

function rpca(D,lambda,rho=1.6,stopCrit1=1.0e-7,stopCrit2=1.0e-5,maxIter=1000)
    #Initialization
    #For problems where we have to add the identity...
    #dims =size(D)
    #if dims[1]==dims[2]
    #    D = D+I
    #end    
    mu = 1.25/norm(D)  
    E = zeros(D)
    k = 0
    Y = D/J(D,lambda)
    A=D
    println("Comienzo!")
    #Iterations
    #In order to avoid evaluation of stoping criteria in the first loop, we do a while true and then 
    #add breaks if the stoping criteria are met.
    while true   
        # First solve A_k+1 = arg min L(A,E_k,Y_k,mu_k)
        singValDesc = svd(D-E+((1/mu)*Y))
        #SVD(A) returns a triple (U,S,V) where S contains the singular values and A=U*S*V' (' denotes ')
        #for SVD(A) to work correctly, A must be m x n where m>= n or else S wont have the proper dimensions.
        A = singValDesc[1]*perturb(diagm(singValDesc[2]),1/mu)*singValDesc[3]'
        #Now solve E_k+1= arg min L(A_k+1,E,Y_k,mu_k)
        perturbFactor = lambda*(1/mu)
        Eupdated = perturb(D-A+((1/mu)*Y),perturbFactor)
        Y = Y + mu*(D-A-Eupdated)
        mu=updateMu(mu,rho,E,Eupdated,D,stopCrit2)
        
       #Checks if both the first and second criterium are met.
        if  firstCriterium(stopCrit1,D,A,Eupdated) && secondCriterium(stopCrit2,D,mu,E,Eupdated) 
            println("Done")
            break
        end
        E=Eupdated
        k=k+1
        println("Step $(k)")
        
        
        #forces the algorithm to stop if k>maxIter. This should never happen. maxIter is 1000 by default.
        if k>maxIter
            break
        end
    end
    return(A,E)
end


rpca (generic function with 5 methods)

In [2]:
#Stoping criteria functions  

function firstCriterium(stopCrit1,D,A,E)
    
    if vecnorm(D-A-E)/vecnorm(D)<stopCrit1
        return true
    end
    return false
end 

#Ek is the value of E computed on the kth step. Ek1 is the value of E computed in the k+1th step.
function secondCriterium(stopCrit2,D,mu,Ek,Ek1)
    
    if (mu*vecnorm(Ek1-Ek))/vecnorm(D)<stopCrit2
        return true
    end
    return false
end


    


secondCriterium (generic function with 1 method)

In [3]:
# Updating functions
#Ek is the value of E computed on the kth step. Ek1 is the value of E computed in the k+1th step.
function updateMu(mu,rho,Ek,Ek1,D,epsilon2)
    
    eval = min(mu,sqrt(mu))*(vecnorm(Ek1-Ek) /vecnorm(D))
    if eval<epsilon2
        return rho*mu
    end
    return mu
end






updateMu (generic function with 1 method)

In [4]:
#Other useful operators

#perturbation operator. Perturbs every entry of the matrix W using the function f.
#W is a m x n matriz to perturb
#epsilon is the perturbation
function perturb(W,perturbation)
    map(W) do x
        f(x,perturbation)
    end
end


#J operator
#W is an m x n matriz
#lambda : parameter of the robust pca
function J(W,lambda)
    return max(norm(W,2),(1/lambda)*maximum(abs(W)))
end

#Method for perturbing x given epsilon. Used in the perturb function.
function f(x,epsilon)
    if x>epsilon 
        return x-epsilon
    end
    if x<-epsilon
        return x+epsilon
    else
        return(0)
    end
end




f (generic function with 1 method)

In [ ]:
#Ejemplo de descomposicion en valores singulares extrana....
asdf = [3 2 2 ; 2 3 -2]

In [5]:
#pruebas
M = [1 1 0 0 1 0; 1 1 0 0 0 0 ; 0 0 1 1 0 1 ; 0 0 1 1 0 0; 1 0 0 0 1 1; 0 0 0 0 1 1]


6×6 Array{Int64,2}:
 1  1  0  0  1  0
 1  1  0  0  0  0
 0  0  1  1  0  1
 0  0  1  1  0  0
 1  0  0  0  1  1
 0  0  0  0  1  1

In [6]:
respuesta=rpca(M,1)

Comienzo!
Step 1
Step 2
Step 3
Done


(
Real[1.0 1.0 … 1.0 -6.245e-17; 1.0 1.0 … -9.97466e-16 -5.55112e-16; … ; 1.0 -1.08247e-15 … 1.0 1.0; -3.88578e-16 -9.4369e-16 … 1.0 1.0],

[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])

In [7]:
respuesta[1]

6×6 Array{Real,2}:
  1.0           1.0          -1.5439e-16   …   1.0          -6.245e-17  
  1.0           1.0           4.35416e-16     -9.97466e-16  -5.55112e-16
  9.71445e-17  -8.60423e-16   1.0             -3.40006e-16   1.0        
 -4.16334e-17  -3.05311e-16   1.0             -8.67362e-17   3.95517e-16
  1.0          -1.08247e-15  -2.94903e-16      1.0           1.0        
 -3.88578e-16  -9.4369e-16   -3.1225e-16   …   1.0           1.0        

In [8]:
respuesta[2]

6×6 Array{Int64,2}:
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0